In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import bernoulli
import seaborn as sns

In [6]:
num_visible_neurons = 10
num_hidden_neurons = 10
num_neurons = num_visible_neurons + num_hidden_neurons
num_timebins = 300
num_iter = 100
print_iter = 10

tau = 0.01
eta_0 = 0.1
tau_adapt = 0.01
rho_0 = 1
v = 0
delta_u = 1
mu_M = 1e-5
mu_Q = 0.00001
tau_G = 0.01
tau_baseline = 0.1
dt = 0.001

## Generate synthetic data
data = np.random.binomial(n = 1, p = 0.1, size = [num_neurons, num_timebins])
true_weights = np.random.normal(loc=0, scale=0.1, size = [num_neurons, num_neurons]) # not used currently, but one could do a simulation to see if the true weights are recovered

## Initialize arrays
potentials = np.zeros((num_neurons, num_timebins))



In [8]:
W_weights = np.random.normal(loc=0, scale=0.1, size = [num_neurons, num_neurons])
Q_weights = np.random.normal(loc=0, scale=0.1, size = [num_neurons, num_neurons])

Q_weights[:, :num_visible_neurons] = np.nan